<a href="https://colab.research.google.com/github/StevenJokess/RL-Adventure/blob/master/test_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/thu-ml/tianshou

  Cloning https://github.com/thu-ml/tianshou to /tmp/pip-req-build-_d0g_2_6
  Running command git clone -q https://github.com/thu-ml/tianshou /tmp/pip-req-build-_d0g_2_6
  Created wheel for tianshou: filename=tianshou-0.3.0-cp36-none-any.whl size=88015 sha256=196fde0c5ddc15ee56b00651e2af5dd0588f9861e37095f2b791b0ea72110f57
  Stored in directory: /tmp/pip-ephem-wheel-cache-1zn94fzo/wheels/da/99/6f/185e522cbaaa133463d09640f64f34f04eda13714f3d5ef0a4
Successfully built tianshou


In [2]:
import gym, torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts

In [3]:
task = 'CartPole-v0'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 8, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, collect_per_step = 1000, 10
writer = SummaryWriter('log/dqn')  # tensorboard is also supported!

In [4]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(test_num)])

In [5]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/latest/tutorials/dqn.html#build-the-network
env = gym.make(task)
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(layer_num=2, state_shape=state_shape, action_shape=action_shape)
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [6]:
policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.ReplayBuffer(buffer_size))
test_collector = ts.data.Collector(policy, test_envs)

In [7]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, collect_per_step,
    test_num, batch_size,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    writer=writer, task=task)
print(f'Finished training! Use {result["duration"]}')

TypeError: ignored

In [8]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, collect_per_step,
    test_num, batch_size,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    writer=writer)
print(f'Finished training! Use {result["duration"]}')

Epoch #1:  47%|####6     | 469/1000 [00:09<00:10, 49.33it/s, env_step=5874, len=200.00, n/ep=1.00, n/st=200.00, rew=200.00, rew_std=0.00, v/ep=70.66, v/st=14131.04]

Finished training! Use 9.51s
